# Process Data
After digging through closed issues on the E-graphsage repo, I saw the author point to this dataset when someone asked where is the `bot.csv` data. There needs to be some processing to combine the files, and append the correct feature names. Here I am bringing everything together so we have one source file.

Original Data Source: https://cloudstor.aarnet.edu.au/plus/s/umT99TnxvbpkkoE?path=%2FCSV%2FEntire%20Dataset

In [1]:
import socket
import struct
import random

import dask.dataframe as dd
from dask_ml.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import networkx as nx

In [2]:
feat_names = pd.read_csv('UNSW_2018_IoT_Botnet_Dataset_Feature_Names.csv')
feat_names = [s.strip() for s in feat_names]

# Some dport and sport values aren't ints, but appear to be hex eg. 00xE or <NA>, so I'm forcing all strings
ddf = dd.read_csv(f'data/*.csv', header=None, names=feat_names, dtype={'dport': 'object', 'sport': 'object'})

In [3]:
ddf.compute()

,pkSeqID,stime,flgs,proto,saddr,sport,daddr,dport,pkts,bytes,...,spkts,dpkts,sbytes,dbytes,rate,srate,drate,attack,category,subcategory
0,1,1.526344e+09,e,arp,192.168.100.1,<NA>,192.168.100.3,<NA>,4,240,...,2,2,120,120,0.002508,0.000836,0.000836,0,Normal,Normal
1,2,1.526344e+09,e,tcp,192.168.100.7,139,192.168.100.4,36390,10,680,...,5,5,350,330,0.006190,0.002751,0.002751,0,Normal,Normal
2,3,1.526344e+09,e,udp,192.168.100.149,51838,27.124.125.250,123,2,180,...,1,1,90,90,20.590960,0.000000,0.000000,0,Normal,Normal
3,4,1.526344e+09,e,arp,192.168.100.4,<NA>,192.168.100.7,<NA>,10,510,...,5,5,210,300,0.006189,0.002751,0.002751,0,Normal,Normal
4,5,1.526344e+09,e,udp,192.168.100.27,58999,192.168.100.1,53,4,630,...,2,2,174,456,0.005264,0.001755,0.001755,0,Normal,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322336,8999996,1.528083e+09,e s,tcp,192.168.100.150,5701,192.168.100.6,80,5,676,...,4,1,616,60,0.067886,0.050914,0.000000,1,DoS,TCP
322337,8999997,1.528083e+09,e s,tcp,192.168.100.150,5706,192.168.100.6,80,4,616,...,4,0,616,0,0.050914,0.050914,0.000000,1,DoS,TCP
322338,8999998,1.528083e+09,e s,tcp,192.168.100.150,5707,192.168.100.6,80,4,616,...,4,0,616,0,0.050914,0.050914,0.000000,1,DoS,TCP
322339,8999999,1.528083e+09,e s,tcp,192.168.100.150,5710,192.168.100.6,80,4,616,...,4,0,616,0,0.050914,0.050914,0.000000,1,DoS,TCP


#### Current Features

In [5]:
ddf.columns

Index(['pkSeqID', 'stime', 'flgs', 'proto', 'saddr', 'sport', 'daddr', 'dport',
       'pkts', 'bytes', 'state', 'ltime', 'seq', 'dur', 'mean', 'stddev',
       'smac', 'dmac', 'sum', 'min', 'max', 'soui', 'doui', 'sco', 'dco',
       'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'srate', 'drate',
       'attack', 'category', 'subcategory'],
      dtype='object')

In [6]:
ddf.to_parquet('data/complete/bot.parquet')

In [10]:
ddf.to_csv('data/complete/bot.csv', single_file=True)

['/home/nfx/Documents/Capstone/data/complete/bot.csv']

-------------
## Data Adjustments
After trying to use the exported parquet file with the existing `E-GraphSAGE-BoT-IoT-mean-binary.ipynb` file from the standard folder in the E-GraphSAGE repo, I noticed we have missing columns that seem to come from additional processing.

**Missing**
- Some source and destinations have `<NA>`. Based on the output in the original notebook, these might have been changed to `-1`?
- They have `state_number` which is int values, while the our current parquet only has `state` with string values.
- They also have `proto_number` we have only `proto` with string values.
- They have `flgs_number` we have `flgs` with strings.

To address these, I will check if this dataset has -1 values for port. If they don't I will replace `<NA>` with `-1`. In regards to the `*_number` columns, I will generate a dictionary and replace the values with a specific number. 

In [2]:
# Using pandas because Dask wouldn't correctly replace '<NA>` values
df = pd.read_csv('data/complete/bot.csv')

/tmp/ipykernel_81851/989524179.py:2: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/complete/bot.csv')


#### Check Ports for -1
Checked for -1, and there weren't any, so I am going to assume -1 is NA for this dataset. They are updated below.

In [3]:
len(df.loc[df['sport'] == '-1']), len(df.loc[df['dport'] == '-1'])

(0, 0)

In [4]:
df[['sport', 'dport']] = df[['sport', 'dport']].fillna('-1')

In [5]:
len(df.loc[df['sport'] == '-1']), len(df.loc[df['dport'] == '-1'])

(2800, 2800)

#### Generate Dicts
Create a int mapping for each state, proto, flgs value.

In [6]:
state_dict = {v: k for k, v in dict(enumerate(df.state.unique())).items()}
proto_dict = {v: k for k, v in dict(enumerate(df.proto.unique())).items()}
flgs_dict = {v: k for k, v in dict(enumerate(df.flgs.unique())).items()}

df['state_number'] = df['state'].map(state_dict)
df['proto_number'] = df['proto'].map(proto_dict)
df['flgs_number'] = df['flgs'].map(flgs_dict)

In [7]:
state_dict

{'CON': 0,
 'INT': 1,
 'RST': 2,
 'FIN': 3,
 'URP': 4,
 'NRS': 5,
 'REQ': 6,
 'PAR': 7,
 'ECO': 8,
 'ACC': 9,
 'TST': 10,
 'MAS': 11,
 'CLO': 12,
 'RSP': 13}

In [8]:
proto_dict

{'arp': 0, 'tcp': 1, 'udp': 2, 'icmp': 3, 'ipv6-icmp': 4, 'igmp': 5, 'rarp': 6}

In [9]:
flgs_dict

{'e': 0,
 'e d': 1,
 'eU': 2,
 'e &': 3,
 'e s': 4,
 'e *': 5,
 'e g': 6,
 'e  D': 7,
 'e dD': 8,
 'e    F': 9,
 'e r': 10,
 'e   t': 11,
 'e dS': 12}

In [14]:
df.to_csv('data/processed/bot.csv')

--------------------------
### E-GraphSAGE Notebook Processing
I'm going to clean up the processing done in the start of the notebook provided with the paper. There is a lot of sloppy work, and perhaps this way I can document step by step.

In [2]:
# Adding drops here because data is large
df = pd.read_csv('data/processed/bot.csv')

/tmp/ipykernel_84996/808290223.py:2: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/processed/bot.csv')


There isn't a clear reason that they dropped some of the features. Some are repeats after creating the dictionary mapping.

In [3]:
# Drop columns, rename
df.drop(columns=['subcategory',
                 'pkSeqID',
                 'stime',
                 'flgs',
                 'category',
                 'state',
                 'proto',
                 'seq',
                 'Unnamed: 0.1',
                 'Unnamed: 0'], inplace=True)

df.rename(columns={'attack': 'label'}, inplace=True)
df.label.value_counts()

label
1    72360933
0        9510
Name: count, dtype: int64

Now we just randomize the source IP addresses, and then append the column so that the formate is `<IP>:<PORT>`.

In [4]:
# Force IP and port number to str
df[['saddr', 'sport', 'daddr', 'dport']] = df[['saddr', 'sport', 'daddr', 'dport']].astype(str)

# Randomly assign new source IP address per the paper
df['saddr'] = df.saddr.apply(
    lambda x: socket.inet_ntoa(struct.pack('>I', random.randint(0xac100001, 0xac1f0001)))
)

# Append port numbers to address then drop port columns
df['saddr'] = df['saddr'] + ':' + df['sport']
df['daddr'] = df['daddr'] + ':' + df['dport']
df.drop(columns=['sport', 'dport'], inplace=True)

#### Ground Truth
Now we are going to store the ground truth labels along with the source and destination IP. This is used later for stratifying our train test split.

In [5]:
label_ground_truth = df[['saddr', 'daddr', 'label']]

Below I am going to create indicator variables. They use the get_dummies in the original notebook, but based on the dataframe output they have, there are fewer unqiue values in their data. You can see that they have proto_state from $[1-5]$, but we will have six. I'm unable to find any information as to why these are different. My best guess is that our data contain almost 19x the number of observations/packets. Again, I'm not able to find information as to why that is true... nor how they might have reduced it.

In [6]:
df = pd.get_dummies(df, columns=['state_number', 'proto_number', 'flgs_number'])

In [7]:
df = df.reset_index()

# Replace infinity values in the dataset. Not sure why we would have them?
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(0, inplace=True)
df.drop(columns=['index'], inplace=True)

In [8]:
df.shape, df.columns

((72370443, 59),
 Index(['saddr', 'daddr', 'pkts', 'bytes', 'ltime', 'dur', 'mean', 'stddev',
        'smac', 'dmac', 'sum', 'min', 'max', 'soui', 'doui', 'sco', 'dco',
        'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'srate', 'drate', 'label',
        'state_number_0', 'state_number_1', 'state_number_2', 'state_number_3',
        'state_number_4', 'state_number_5', 'state_number_6', 'state_number_7',
        'state_number_8', 'state_number_9', 'state_number_10',
        'state_number_11', 'state_number_12', 'state_number_13',
        'proto_number_0', 'proto_number_1', 'proto_number_2', 'proto_number_3',
        'proto_number_4', 'proto_number_5', 'proto_number_6', 'flgs_number_0',
        'flgs_number_1', 'flgs_number_2', 'flgs_number_3', 'flgs_number_4',
        'flgs_number_5', 'flgs_number_6', 'flgs_number_7', 'flgs_number_8',
        'flgs_number_9', 'flgs_number_10', 'flgs_number_11', 'flgs_number_12'],
       dtype='object'))

In [9]:
df.head()

,saddr,daddr,pkts,bytes,ltime,dur,mean,stddev,smac,dmac,...,flgs_number_3,flgs_number_4,flgs_number_5,flgs_number_6,flgs_number_7,flgs_number_8,flgs_number_9,flgs_number_10,flgs_number_11,flgs_number_12
0,172.20.116.142:-1,192.168.100.3:-1,4,240,1.526345e+09,1195.996582,0.000006,0.000002,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1,172.23.198.15:139,192.168.100.4:36390,10,680,1.526346e+09,1453.945923,0.000028,0.000008,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
2,172.18.144.36:51838,27.124.125.250:123,2,180,1.526344e+09,0.048565,0.048565,0.000000,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
3,172.19.40.134:-1,192.168.100.7:-1,10,510,1.526346e+09,1454.080322,0.000238,0.000022,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
4,172.16.213.21:58999,192.168.100.1:53,4,630,1.526345e+09,569.933960,0.098505,0.080150,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False


At this point in the paper notebook, the dataset is `(3668522, 58)` while ours is `(72370443, 59)`. The extra column is most likely an additonal protocol in our data, but I can't confirm that statement. We also have many many more observations.

In [10]:
df.to_parquet('data/processed/bot.parquet')

The section below gets a little messy. Despite having 164gb memory, I wasn't able to perform scaling on the data without switching back to Dask. Then, when I tried to split the data, the Dask-ML version of train_test_split doesn't support straification. This forced me back to Pandas.

In [2]:
ddf = dd.read_parquet('data/processed/bot.parquet')

bool_columns = [
    'state_number_0', 'state_number_1', 'state_number_2', 'state_number_3', 'state_number_4',
    'state_number_5', 'state_number_6', 'state_number_7', 'state_number_8', 'state_number_9',
    'state_number_10', 'state_number_11', 'state_number_12', 'state_number_13',
    'proto_number_0', 'proto_number_1', 'proto_number_2', 'proto_number_3', 'proto_number_4',
    'proto_number_5', 'proto_number_6',
    'flgs_number_0', 'flgs_number_1', 'flgs_number_2', 'flgs_number_3', 'flgs_number_4',
    'flgs_number_5', 'flgs_number_6', 'flgs_number_7', 'flgs_number_8', 'flgs_number_9',
    'flgs_number_10', 'flgs_number_11', 'flgs_number_12'
]

# Convert boolean columns to integers
ddf[bool_columns] = ddf[bool_columns].astype(int)

In [4]:
# Scale the data and split for export
scaler = StandardScaler()
cols_to_norm = list(set(list(ddf.iloc[:, 2:].columns )) - set(list(['label'])))
ddf_scaled = scaler.fit_transform(ddf[cols_to_norm])

Save the scaler for inference.

In [7]:
import pickle
with open('data/processed/scaler.pkl','wb') as f:
    pickle.dump(scaler, f)

In [8]:
df = pd.read_parquet('data/processed/bot.parquet')

In [11]:
label_ground_truth = df[['saddr', 'daddr', 'label']]

X_train, X_test, y_train, y_test = train_test_split(
     df, label_ground_truth, test_size=0.3, random_state=42, stratify=label_ground_truth.label
)

In [12]:
X_train.to_parquet('data/final/x_train.parquet')
X_test.to_parquet('data/final/x_test.parquet')
y_train.to_parquet('data/final/y_train')
y_test.to_parquet('data/final/y_test')

-----------
### Generate Graphs

In [2]:
# Reload from disk to free memory.
X_train = pd.read_parquet('data/final/x_train.parquet')
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns )) - set(list(['label'])))


X_train['h'] = X_train[cols_to_norm].values.tolist()
G = nx.from_pandas_edgelist(X_train, "saddr", "daddr", ['h','label'], create_using= nx.MultiGraph())

The above graph fills up 90% of my memory. Forming the directed graph causes it to hit 100% and kill the kernel.

In [ ]:
G = G.to_directed()
G = from_networkx(G,edge_attrs=['h','label'])